#### https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

train_df = pd.read_csv('/Users/babamalik/Desktop/Data_Scientist/Store Sales - Time Series Forecasting/data/train.csv', parse_dates=["date"])
test_df = pd.read_csv("/Users/babamalik/Desktop/Data_Scientist/Store Sales - Time Series Forecasting/data/test.csv", parse_dates=["date"])
stores_df = pd.read_csv("/Users/babamalik/Desktop/Data_Scientist/Store Sales - Time Series Forecasting/data/stores.csv")
oil_df = pd.read_csv("/Users/babamalik/Desktop/Data_Scientist/Store Sales - Time Series Forecasting/data/oil.csv", parse_dates=["date"])
holidays_df = pd.read_csv("/Users/babamalik/Desktop/Data_Scientist/Store Sales - Time Series Forecasting/data/holidays_events.csv", parse_dates=["date"])
transactions_df = pd.read_csv("/Users/babamalik/Desktop/Data_Scientist/Store Sales - Time Series Forecasting/data/transactions.csv", parse_dates=["date"])

In [51]:
# Display basic info
print(train_df.head())
print(train_df.info())

   id       date  store_nbr      family  sales  onpromotion
0   0 2013-01-01          1  AUTOMOTIVE    0.0            0
1   1 2013-01-01          1   BABY CARE    0.0            0
2   2 2013-01-01          1      BEAUTY    0.0            0
3   3 2013-01-01          1   BEVERAGES    0.0            0
4   4 2013-01-01          1       BOOKS    0.0            0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 137.4+ MB
None


In [52]:
train_df['family'].nunique()
# 'nunique()': Use this method when you only need the count of unique values.

33

In [53]:
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          83488 non-null  datetime64[ns]
 1   store_nbr     83488 non-null  int64         
 2   transactions  83488 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.9 MB


In [54]:
# Merge transactions into train data
train_df = train_df.merge(transactions_df, on=["date", "store_nbr"], how="left")

In [55]:
train_df['transactions'].count()

np.int64(2755104)

In [56]:
train_df['transactions'].isna().sum()

np.int64(245784)

In [57]:
len(train_df['transactions'])

3000888

In [58]:
train_df.head()

,id,date,store_nbr,family,sales,onpromotion,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,NaN


In [59]:
# Fill missing transaction values (if any) with the median
train_df["transactions"].fillna(train_df["transactions"].median(), inplace=True)

/var/folders/nv/hf6t5yfd0_75lf4kc1jtwh7m0000gn/T/ipykernel_34037/187478240.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df["transactions"].fillna(train_df["transactions"].median(), inplace=True)


In [60]:
# Check if the merge was successful
print("Merged Data:\n", train_df.head())

Merged Data:
    id       date  store_nbr      family  sales  onpromotion  transactions
0   0 2013-01-01          1  AUTOMOTIVE    0.0            0        1393.0
1   1 2013-01-01          1   BABY CARE    0.0            0        1393.0
2   2 2013-01-01          1      BEAUTY    0.0            0        1393.0
3   3 2013-01-01          1   BEVERAGES    0.0            0        1393.0
4   4 2013-01-01          1       BOOKS    0.0            0        1393.0


In [61]:
train_df = train_df.merge(oil_df, on="date", how="left")

In [62]:
train_df.head()

,id,date,store_nbr,family,sales,onpromotion,transactions,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,1393.0,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,1393.0,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,1393.0,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,1393.0,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,1393.0,NaN


In [69]:
# Fill missing oil prices using forward fill
train_df["dcoilwtico"] = train_df["dcoilwtico"].ffill()
 # Fill missing oil prices

In [70]:
train_df.head()

,id,date,store_nbr,family,sales,onpromotion,transactions,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,1393.0,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,1393.0,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,1393.0,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,1393.0,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,1393.0,NaN


In [71]:
print(train_df["dcoilwtico"].isnull().sum()) 

1782


In [76]:
train_df["dcoilwtico"] = train_df["dcoilwtico"].ffill().bfill()

In [77]:
print(train_df["dcoilwtico"].isnull().sum()) 

0


In [72]:
# Merge holidays data
train_df = train_df.merge(holidays_df[["date", "type"]], on="date", how="left")

In [74]:
train_df["type"] = train_df["type"].fillna("No Holiday")

In [75]:
print(train_df["type"].isnull().sum())

0


In [78]:
train_df.head()

,id,date,store_nbr,family,sales,onpromotion,transactions,dcoilwtico,type
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,1393.0,93.14,Holiday
1,1,2013-01-01,1,BABY CARE,0.0,0,1393.0,93.14,Holiday
2,2,2013-01-01,1,BEAUTY,0.0,0,1393.0,93.14,Holiday
3,3,2013-01-01,1,BEVERAGES,0.0,0,1393.0,93.14,Holiday
4,4,2013-01-01,1,BOOKS,0.0,0,1393.0,93.14,Holiday


In [79]:
# Merge store metadata
train_df = train_df.merge(stores_df, on="store_nbr", how="left")

In [81]:
# Check final merged dataset
train_df.head()

,id,date,store_nbr,family,sales,onpromotion,transactions,dcoilwtico,type_x,city,state,type_y,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,1393.0,93.14,Holiday,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.0,0,1393.0,93.14,Holiday,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.0,0,1393.0,93.14,Holiday,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.0,0,1393.0,93.14,Holiday,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.0,0,1393.0,93.14,Holiday,Quito,Pichincha,D,13


In [83]:
unique_values = pd.concat([train_df['type_x'], train_df['type_y']]).unique()

In [86]:
unique_values

array(['Holiday', 'No Holiday', 'Work Day', 'Additional', 'Event',
       'Transfer', 'Bridge', 'D', 'C', 'B', 'E', 'A'], dtype=object)

# Data Preprocessing